# Detecção de Fraude em Cartões de Crédito - Notebook Organizacional
## Introdução
**Objetivo:**
Neste notebook, iremos construir um projeto completo de detecção de fraudes em transações de cartão de crédito utilizando técnicas de Deep Learning. O foco é explicar conceitos de forma didática, aplicando boas práticas de projeto.

**Por que é importante?**
Fraudes em cartões de crédito geram prejuízos significativos. Modelos de detecção automatizados auxiliam a identificar transações suspeitas antes que causem danos.

**Como funciona em termos gerais?**
A ideia central é treinar uma rede neural para aprender padrões presentes em transações legítimas e fraudulentas. Cada transação possui diversas características (features) que alimentam o modelo. Assim como um filtro que aprende a separar materiais de acordo com suas propriedades, a rede neural refina as informações em camadas, ajustando pesos para classificar corretamente.

### Estrutura do Notebook
1. Importações e Verificação de Versões
2. Carregamento e Exploração dos Dados (EDA)
3. Pré-processamento: Divisão, Padronização e Tratamento de Desbalanceamento
4. Definição da Arquitetura da Rede Neural
5. Treinamento com Callbacks e Class Weights
6. Avaliação do Modelo: Métricas e Visualizações
7. Função de Previsão para Novas Transações
8. Exportação de Modelo e Scaler
9. Conclusões e Próximos Passos


## 1. Importações e Verificação de Versões
**O que será feito:**
Importaremos bibliotecas essenciais para manipulação de dados, visualizações e construção do modelo.

**Por que é importante?**
Verificar versões garante compatibilidade e reprodutibilidade do projeto.


In [ ]:
# Importando bibliotecas essenciais
import sys  # Para verificar versão do Python
import pandas as pd  # Manipulação de dados
import numpy as np  # Operações numéricas
import matplotlib.pyplot as plt  # Visualizações básicas
import seaborn as sns  # Visualizações avançadas
import tensorflow as tf  # Framework de Deep Learning
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    confusion_matrix, classification_report,
    roc_auc_score, RocCurveDisplay,
    precision_recall_curve, auc
)
from imblearn.over_sampling import SMOTE  # Tratamento de desbalanceamento
import joblib  # Salvamento de objetos Python
import sklearn  # Biblioteca de aprendizado de máquina

# Exibindo versões para garantia de reprodutibilidade
print("Python:", sys.version)
print("Pandas:", pd.__version__)
print("NumPy:", np.__version__)
print("TensorFlow:", tf.__version__)
print("scikit-learn:", sklearn.__version__)
print("imbalanced-learn:", SMOTE.__module__.split('.')[0])

# Configurando estilo das figuras
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

# Semente para reprodutibilidade
np.random.seed(42)


## 2. Carregamento e Exploração dos Dados (EDA)
**O que será feito:**
1. Leremos o arquivo CSV com as transações de cartão de crédito.
2. Faremos uma análise exploratória inicial, visualizando a distribuição das classes e estatísticas descritivas.

**Por que é importante?**
Compreender a estrutura dos dados e o grau de desbalanceamento é crucial para planejar o modelo e as estratégias de pré-processamento.


In [ ]:
# 3.1. Leitura do CSV
df = pd.read_csv('creditcard.csv')  # Assegure-se de ter enviado o arquivo para o Colab
print("Formato dos dados:", df.shape)
df.head(5)


### 2.2. Distribuição da Classe (Fraude vs. Não-Fraude)
**O que será feito:**
Visualizar quantidade de transações legítimas e fraudulentas. Calcular proporção.

**Por que é importante?**
O dataset é altamente desbalanceado, e entender essa proporção ajuda a decidir técnicas de tratamento.


In [ ]:
# Plot da distribuição das classes
plt.figure(figsize=(6,4))
sns.countplot(x='Class', data=df)
plt.title("Distribuição entre Transações Legítimas (0) e Fraudulentas (1)")
plt.xticks([0,1], ['Legítima', 'Fraude'])
plt.show()

# Proporção de cada classe
proporcao = df['Class'].value_counts(normalize=True)
print("Proporção de cada classe:\n", proporcao)


### 2.3. Estatísticas Descritivas de ‘Time’ e ‘Amount’ e Visualizações
**O que será feito:**
Exibir estatísticas descritivas e analisar distribuições de ‘Amount’. Visualizar boxplot e histograma.

**Por que é importante?**
Variáveis como ‘Amount’ podem ter distribuições assimétricas e outliers que impactam o modelo.


In [ ]:
# Estatísticas descritivas de ‘Time’ e ‘Amount’
df[['Time','Amount']].describe().round(2)

# Histograma de 'Amount'
plt.figure(figsize=(6,4))
sns.histplot(df['Amount'], bins=50, kde=True)
plt.title("Distribuição do Valor (‘Amount’)")
plt.xlabel("Montante (em USD)")
plt.show()

# Boxplot de 'Amount' por Classe (escala log para legibilidade)
plt.figure(figsize=(6,4))
sns.boxplot(x='Class', y='Amount', data=df)
plt.yscale('log')  # Torna a escala mais legível para outliers
plt.title("Boxplot de Montante por Classe")
plt.xticks([0,1], ['Legítima', 'Fraude'])
plt.show()

# Resumo de um subconjunto de features PCA (V1 a V28) para ilustração
df[['V1','V2','V3','V28']].describe().round(2)


## 3. Pré-processamento: Divisão, Padronização e Tratamento de Desbalanceamento
### 3.1. Definir X e y
**O que será feito:**
Separar features (`X`) e variável alvo (`y`). Verificar e remover possíveis NaNs em `y`.

**Por que é importante?**
Garantir que não existam valores ausentes na variável alvo evita erros no treinamento.


In [ ]:
# Separando features e target
X = df.drop('Class', axis=1).copy()
y = df['Class'].copy()

# Verificar e remover linhas com NaN em y (precaução)
if y.isnull().any():
    print("NaN values found in y. Removing corresponding rows.")
    nan_indices = y[y.isnull()].index
    X = X.drop(nan_indices)
    y = y.drop(nan_indices)
    print(f"Removed {len(nan_indices)} rows containing NaN in y.")


### 3.2. Divisão em Treino e Teste (com Stratify)
**O que será feito:**
Dividiremos o dataset em 80% treino e 20% teste, mantendo a proporção das classes (`stratify`).

**Por que é importante?**
Manter a proporção de classes em ambos conjuntos evita viés na avaliação.


In [ ]:
# Dividindo em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.20,
    random_state=42,
    stratify=y
)
print("Shape treino:", X_train.shape, "Shape teste:", X_test.shape)


### 3.3. Padronizar Features Numéricas
**O que será feito:**
Padronizar todos os atributos (incluindo ‘Time’ e ‘Amount’) para melhorar a convergência do modelo.

**Por que é importante?**
Redes neurais são sensíveis às escalas das features; normalizar ajuda no treinamento estável.


In [ ]:
# Inicializando StandardScaler
scaler = StandardScaler()

# Ajustar e transformar o conjunto de treino; transformar o conjunto de teste
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# Salvando scaler para uso posterior
joblib.dump(scaler, 'scaler.pkl')

# Verificação rápida das estatísticas após padronização
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X.columns)
print("Média (treino):", X_train_scaled_df.mean().round(2).head())
print("Std (treino):", X_train_scaled_df.std().round(2).head())


### 3.4. Análise do Desbalanceamento
**O que será feito:**
Contar transações legítimas vs. fraudulentas no conjunto de treino.

**Por que é importante?**
Taxa de fraude muito baixa (cerca de 0.2%) exige estratégias específicas para evitar que o modelo aprenda apenas a classe majoritária.


In [ ]:
# Contagem de classes no conjunto de treino
contagem = y_train.value_counts()
print("Contagem no treino:\n", contagem)
print(f"Taxa de fraude no treino: {contagem[1]/(contagem[0]+contagem[1]):.4f}")


### 3.5. Estratégias para Lidar com Desbalanceamento
**Opções abordadas:**
1. `class_weight`: penalizar mais os erros na classe minoritária.
2. `SMOTE`: gerar amostras sintéticas da classe minoritária.

**Por que é importante?**
Em problemas de fraude, queremos maximizar a detecção de fraudes (alta `recall` para classe 1), sem gerar muitos falsos positivos.


In [ ]:
# 3.5.1. Cálculo de class_weight inversamente proporcional à frequência
weight_for_0 = (1 / contagem[0]) * (len(y_train) / 2.0)
weight_for_1 = (1 / contagem[1]) * (len(y_train) / 2.0)
class_weights = {0: weight_for_0, 1: weight_for_1}
print("Class weights:", class_weights)

# 3.5.2. Oversampling com SMOTE (opcional)
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train_scaled, y_train)
print("Após SMOTE, nova contagem:", pd.Series(y_train_res).value_counts())

# Observação: Escolha entre usar class_weight ou SMOTE. Comparar resultados posteriormente.


## 4. Definição da Arquitetura da Rede Neural (Keras)
### 4.1. Configuração do Modelo
**O que será feito:**
Definiremos um modelo `Sequential` com camadas `Dense` e `Dropout`. Incluiremos uma camada adicional para representação intermediária.

**Por que é importante?**
Arquitetura de rede define a capacidade do modelo em aprender padrões complexos. Camadas `Dropout` ajudam a prevenir overfitting.


In [ ]:
# Definindo dimensão de entrada
input_dim = X_train_scaled.shape[1]  # Número de features

# Construindo o modelo
model = Sequential([
    tf.keras.Input(shape=(input_dim,)),  # Camada de entrada explicita
    Dense(32, activation='relu'),  # Primeira camada oculta com 32 neurônios
    Dropout(0.3),  # Dropout de 30% para reduzir overfitting
    Dense(16, activation='relu'),  # Segunda camada oculta com 16 neurônios
    Dropout(0.2),  # Dropout de 20%
    Dense(8, activation='relu'),   # Terceira camada oculta com 8 neurônios
    Dropout(0.2),  # Dropout de 20%
    Dense(1, activation='sigmoid')  # Camada de saída para classificação binária
])

# Compilando o modelo com métricas apropriadas para desbalanceamento
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc')  # AUC-ROC
    ]
)

# Exibindo resumo da arquitetura
model.summary()


## 5. Treinamento com Callbacks e Class Weights
### 5.1. Configuração de Callbacks
**O que será feito:**
Definiremos `EarlyStopping` para evitar overfitting e `ModelCheckpoint` para salvar o melhor modelo conforme AUC.

**Por que é importante?**
`EarlyStopping` interrompe o treino quando não há melhoria, economizando recursos. `ModelCheckpoint` armazena pesos do melhor modelo.


In [ ]:
# Configurando callbacks
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,  # Para se não houver melhoria em 5 epochs
    restore_best_weights=True
)
chkpt = ModelCheckpoint(
    'best_fraud_model.h5',
    monitor='val_auc',
    mode='max',
    save_best_only=True,
    verbose=1
)


### 5.2. Treinamento do Modelo
**O que será feito:**
Treinaremos usando `class_weight` para penalizar a classe minoritária. Caso prefira SMOTE, basta alterar para `(X_train_res, y_train_res)` e remover `class_weight`.

**Por que é importante?**
Garantir que o modelo não ignore fraudes, valorizando erros em transações fraudulentas.


In [ ]:
# Treinando o modelo com class weights
history = model.fit(
    X_train_scaled, y_train,
    validation_split=0.2,  # 20% do treino para validação
    epochs=50,
    batch_size=2048,
    class_weight=class_weights,  # Penalização para classe minoritária
    callbacks=[early_stop, chkpt],
    verbose=1
)

# Se preferir usar SMOTE, comente o bloco acima e use o abaixo:
# history = model.fit(
#     X_train_res, y_train_res,
#     validation_split=0.2,
#     epochs=50,
#     batch_size=2048,
#     callbacks=[early_stop, chkpt],
#     verbose=1
# )

# Carregando o melhor modelo salvo
best_model = load_model('best_fraud_model.h5')


## 6. Avaliação do Modelo: Métricas e Visualizações
**O que será feito:**
1. Predições no conjunto de teste.
2. Matriz de confusão.
3. Relatório de classificação (precision, recall, f1-score).
4. Curvas ROC AUC e Precision-Recall.

**Por que é importante?**
Avaliar o modelo em dados nunca vistos mostra sua generalização e ajuda a entender trade-offs em desbalanceamento.


In [ ]:
# 6.1. Predições no conjunto de teste
y_prob = best_model.predict(X_test_scaled, batch_size=2048).ravel()
y_pred = (y_prob >= 0.5).astype(int)

# 6.2. Matriz de Confusão
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(5,4))
sns.heatmap(
    cm, annot=True, fmt='d', cmap='Blues',
    xticklabels=['Legítima (0)', 'Fraude (1)'],
    yticklabels=['Legítima (0)', 'Fraude (1)']
)
plt.ylabel('True')
plt.xlabel('Predicted')
plt.title("Matriz de Confusão (Test Set)")
plt.show()

# 6.3. Relatório de Classificação
print("Relatório de Classificação:")
print(classification_report(y_test, y_pred, digits=4))

# 6.4. ROC AUC e Curva ROC
auc_roc = roc_auc_score(y_test, y_prob)
print(f"AUC-ROC (Test): {auc_roc:.4f}")

RocCurveDisplay.from_predictions(y_test, y_prob)
plt.title("Curva ROC (Test)")
plt.show()

# 6.5. Precision-Recall Curve e AUC-PR
precision, recall, thresholds = precision_recall_curve(y_test, y_prob)
auc_pr = auc(recall, precision)
print(f"AUC-PR (Test): {auc_pr:.4f}")

plt.figure(figsize=(6,4))
plt.plot(recall, precision, label=f'AUC-PR = {auc_pr:.4f}')
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Curva Precision-Recall (Test)")
plt.legend()
plt.show()


## 7. Função de Previsão para Novas Transações
**O que será feito:**
Definiremos uma função que recebe um array de features de uma nova transação e retorna probabilidade e classificação com base em um threshold.

**Por que é importante?**
Em produção, precisamos de uma forma simples de usar o modelo treinado em transações ao vivo.


In [ ]:
def predict_transaction(values_array, scaler, model, threshold=0.5):
    """
    Recebe: 
      - values_array: array ou lista com 30 features na mesma ordem do dataframe original
      - scaler: objeto StandardScaler ajustado nos dados de treino
      - model: modelo Keras carregado
      - threshold: ponto de corte para classificar como fraude (padrão 0.5)
    Retorna:
      - dicionário com probabilidade e label ('Fraude' ou 'Não Fraude')
    """
    # Convertendo para numpy array e redimensionando
    arr = np.array(values_array).reshape(1, -1)  # (1, 30)
    # Padronizando os valores
    arr_scaled = scaler.transform(arr)
    # Obtendo probabilidade de fraude
    prob = float(model.predict(arr_scaled).ravel())
    # Definindo rótulo com base no threshold
    label = "Fraude" if prob >= threshold else "Não Fraude"
    return {"probabilidade": prob, "label": label}

# Exemplo de uso da função (usamos a primeira transação de teste)
exemplo = X_test.iloc[0].values
resultado = predict_transaction(exemplo, scaler, best_model)
print(f"Probabilidade: {resultado['probabilidade']:.4f}")
print(f"Classificação: {resultado['label']}")


## 8. Exportação de Modelo e Scaler
**O que será feito:**
Salvaremos o scaler e o modelo final para uso em produção.

**Por que é importante?**
Permite carregar rapidamente o modelo escalado em aplicações sem necessidade de retrinar.


In [ ]:
# O modelo já foi salvo pelo ModelCheckpoint em 'best_fraud_model.h5'
print("Modelo salvo como 'best_fraud_model.h5'.")

# Salvando o scaler (já realizado anteriormente, mas reforçando)
joblib.dump(scaler, 'scaler.pkl')
print("Scaler salvo como 'scaler.pkl'.")


## 9. Conclusões e Próximos Passos
**Conclusões:**
- Utilizamos uma abordagem de rede neural com camadas densas e dropout, obtendo bom desempenho ao priorizar métricas como AUC, precision e recall.
- O uso de `class_weight` e `SMOTE` foram estratégias eficazes para tratar o desbalanceamento, mas precisam ser comparadas empiricamente.
- Ferramentas como `EarlyStopping` e `ModelCheckpoint` garantem melhor controle sobre o treinamento.

**Próximos Passos:**
- Testar arquiteturas mais avançadas, como Autoencoders para detecção de anomalias.
- Implementar pipelines com `tf.data.Dataset` para melhor performance em produção.
- Realizar tuning de hiperparâmetros com `GridSearchCV` ou `RandomizedSearchCV`.
- Documentar em um repositório GitHub com README explicativo, instruções de execução e referências.
